발견된 수술 도구 카테고리: 26개
 ID 1: Bowel Graspers (straight) (AESCULAP) (GRSL-BWAE)
 ID 2: Dissector (Artisential) (DSTL-ATUK)
 ID 3: Dissector (kelly) (AESCULAP) (DSTL-KLAE)
 ID 4: Drain (JPCL-UCUK)
 ID 5: Endobulldog Clamp (CLML-BDUK)
 ID 6: Gauze (GAUL-UCUK)
 ID 7: Gauze Ball (GAUL-BAUK)
 ID 8: Graspers (curved) (KARL STORZ) (GRSL-C2KS)
 ID 9: Graspers (curved) (KARL STORZ) (GRSL-CIKS)
 ID 10: Graspers (duckbill) (GRSL-DBUK)
 ID 11: Graspers (straight) (KARL STORZ) (GRSL-STKS)
 ID 12: Harmonic Scalpel (ETHICON) (HMSL-UCET)
 ID 13: Irrigator (IRGL-UCUK)
 ID 14: Ligasure (COVIDIEN) (LGSL-MACV)
 ID 15: Linear Stapler (EGSL-UCUK)
 ID 16: Linear Stapler (ETHICON) (STPL-UCET)
 ID 17: Liver retractor (LVRL-UCUK)
 ID 18: Needle Holder (AESCULAP) (NDHL-UCAE)
 ID 19: Obturator (OBTL-UCUK)
 ID 20: Polymer Clip Applier (CLAL-UCUK)
 ID 21: Spatula Suction (ETHICON) (SPSL-UCET)
 ID 22: Specimen Retrieval Bag (RTBL-UCUK)
 ID 23: Suction Irrigator (ETHICON) (SIRL-UCET)
 ID 24: Suction Irrigator (RDSL-UCUK)

TypeError: expected string or bytes-like object

In [1]:
# Keras/TF 버전으로 수정: 필요한 라이브러리 임포트
import os
import sys
import json
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

# Keras 및 TensorFlow 관련 라이브러리
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

# lxml 임포트
try:
    import lxml.etree as etree
except ImportError:
    import xml.etree.ElementTree as etree
    print("경고: lxml 라이브러리를 찾을 수 없어 표준 xml.etree.ElementTree를 사용합니다.")

# Matterport Mask R-CNN 라이브러리 임포트
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# --- 1. 데이터 파싱 및 설정 클래스 (이전과 거의 동일) ---
# 이전 답변에서 제공된 SurgicalDatasetParser, SurgicalDataset, SurgicalConfig 클래스를 그대로 사용합니다.
# 생략된 코드는 첨부된 파일 [1] 또는 이전 답변을 참고해주세요.
class SurgicalDatasetParser:
    # ... 이전 코드와 동일 ... [1]
    def __init__(self, base_path, phase_directory_path):
        self.base_path = Path(base_path)
        self.label_path = self.base_path/"라벨링데이터"/"클래스1.(수술도구 이미지)"/"01.원위부위절제술"
        self.image_path = self.base_path/"원천데이터"/"클래스1.(수술도구 이미지)"/"01.원위부위절제술"
        self.tool_categories = self._discover_tool_categories()
        self.class_to_idx = {cat['name']: cat['id'] for cat in self.tool_categories}
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}
        print(f"발견된 수술 도구 카테고리: {len(self.tool_categories)}개")
        for cat in self.tool_categories: print(f" ID {cat['id']}: {cat['name']}")
        self.image_stem_to_phase_map = self._load_phase_data(phase_directory_path)
        if self.image_stem_to_phase_map:
            self.phases = sorted(list(set(self.image_stem_to_phase_map.values())))
            self.phase_to_idx = {name: i for i, name in enumerate(self.phases)}
            self.idx_to_phase = {i: name for i, name in enumerate(self.phases)}
            self.num_phases = len(self.phases)
            print(f"발견된 수술 단계: {self.num_phases}개 - {self.phases}")
        else:
            self.phases, self.phase_to_idx, self.idx_to_phase, self.num_phases = [], {}, {}, 0
    def _discover_tool_categories(self):
        categories = []
        if self.label_path.exists():
            tool_folders = sorted([f for f in self.label_path.iterdir() if f.is_dir()])
            categories = [{"id": idx + 1, "name": folder.name} for idx, folder in enumerate(tool_folders)]
        return categories
    def _load_phase_data(self, phase_directory_path):
        image_stem_to_phase = {}
        if not phase_directory_path: return image_stem_to_phase
        phase_dir = Path(phase_directory_path)
        if not phase_dir.is_dir(): return image_stem_to_phase
        for json_file_path in phase_dir.glob("*.json"):
            try:
                with open(json_file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                for entry in data:
                    if isinstance(entry, dict) and "filename" in entry and "phase" in entry:
                        image_stem_to_phase[Path(entry["filename"]).stem] = entry["phase"]
            except Exception: continue
        return image_stem_to_phase
    def prepare_dataset_for_keras(self):
                                               
        print("데이터셋 준비 시작 :  하위 폴더에서 모든 xmL 파일을 찾습니다.")
        all_xml_files = list(self.label_path.rglob("*.xml"))
        total_items = len(all_xml_files)
                                               
        if total_items == 0:
            print("경고 : 지정된 경로에서 XML 파일을 찾을 수 없습니다.")
            return 
                                               
        print(f"총 {total_items}개의 XML 파일을 찾았습니다.")   
                                               
        processed_count = 0

        for i, xml_file in enumerate(all_xml_files):
            print(f"진행 중 : {i + 1} / {total_items} 항목 처리 완료", end = '\r') 
            tool_dir_name = xml_file.parent.name

            annotation = self._parse_xml_annotation(xml_file)
            if not annotation or not annotation.get('objects'): continue

            image_file = self._find_matching_image(xml_file.stem, tool_dir_name)

            if not image_file: continue

            phase_label = self.image_stem_to_phase_map.get(xml_file.stem)
            if phase_label is None or phase_label not in self.phase_to_idx: continue
            processed_count += 1
                                               
            yield{
                'image_path': str(image_file),
                'width': annotation['size']['width'], 
                'height': annotation['size']['height'],
                'objects': annotation['objects'],
                'phase_id': self.phase_to_idx[phase_label]
            }
                                               
        print("\n" + "="*50)                                               
        print(f"총 {processed_count}개 학습/검증용 아이템 준비 완료")
        print("="*50)       
                                               
        #return dataset_items
                                               
    def _parse_xml_annotation(self, xml_file):
        try:
            tree = etree.parse(str(xml_file))
            root = tree.getroot()
                                               
            size_elem = root.find('size')
            if size_elem is None or size_elem.find('width') is None or size_elem.find('height') is None:
                return None    
            width = int(size_elem.find('width').text)
            height = int(size_elem.find('height').text)
            objects = []
                                               
            for obj_elem in tree.findall('object'):
                class_name_elem = obj_elem.find('name')
                if class_name_elem is None or class_name_elem.text not in self.class_to_idx: continue

                class_name = class_name_elem.text                               
                points_elem = obj_elem.find('points')
                if points_elem is not None:
                    x_coords = points_elem.findall('x')
                    y_coords = points_elem.findall('y')
                    points = [{'x': float(x.text), 'y': float(y.text)} for x, y in zip(x_coords,y_coords)]
                    if points: objects.append({'name': class_name, 'points': points})
            return {'size': {'width': width, 'height': height}, 'objects': objects}
                                               
        except etree.XMLSyntaxError as e: 
            print(f"\n [오류]XML 파일 파싱 오류 {xml_file.name}:{e} 이 파일을 건너 뜁니다. ")                        
            return None
                                               
        except Exception  as e: 
            print(f"\n [오류]알 수 없는 오류 발생 {xml_file.name}:{e} 이 파일을 건너 뜁니다. ")                        
            return None
                                               
    def _find_matching_image(self, stem, folder):
        image_tool_folder = self.image_path / folder           
        if not image_tool_folder.is_dir():
            return None
        possible_extensions = ['.png', '.jpg','.jpeg','.PNG', '.JPG','.JPEG'] 
        try:
                                               
            for ext in possible_extensions:
                img_path = self.image_path / folder / (stem + ext)
                if img_path.exists():
                    return img_path
        except Exception as e:
            print(f"[경고] 디렉토리 접근 오류. 건너뜁니다. 경로: {image_tool_folder} / {stem}(오류 : {e})")
            return None
        return None


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
class SurgicalDataset(utils.Dataset):
    # ... 이전 코드와 동일 ... [1]
    def load_surgical_data(self, parser, dataset_items, dataset_type):
        self.parser = parser
        for cat in self.parser.tool_categories: self.add_class("surgical_tool", cat['id'], cat['name'])
        for i, item in enumerate(dataset_items):
            self.add_image("surgical_tool", image_id=i, path=item['image_path'], width=item['width'], height=item['height'], polygons=item['objects'], phase_id=item['phase_id'])
        print(f"'{dataset_type}' 데이터셋에 {len(dataset_items)}개 이미지를 로드했습니다.")
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        masks, class_ids = [], []
        for obj in info["polygons"]:
            points = np.array([[p['x'], p['y']] for p in obj['points']], dtype=np.int32)
            mask = np.zeros([info["height"], info["width"]], dtype=np.uint8)
            cv2.fillPoly(mask, [points], 1)
            masks.append(mask)
            class_ids.append(self.parser.class_to_idx[obj['name']])
        return (np.stack(masks, axis=-1).astype(bool), np.array(class_ids, dtype=np.int32)) if masks else super(self.__class__, self).load_mask(image_id)
    def image_reference(self, image_id): return self.image_info[image_id]["path"]
    def load_phase_label(self, image_id): return self.image_info[image_id]['phase_id']


In [3]:

class SurgicalConfig(Config):
    # ... 이전 코드와 동일 ... [1]
    NAME = "surgical_tool_phase"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    LEARNING_RATE = 0.001

# --- 2. 모델 학습 및 특징 추출, 분류기 학습 함수 정의 ---

def train_mask_rcnn(config, train_dataset, val_dataset, model_dir, coco_model_path):
    """1단계: Mask R-CNN 모델을 학습시킵니다."""
    print("\n--- 1단계: Mask R-CNN (특징 추출기) 학습 시작 ---")
    model = modellib.MaskRCNN(mode="training", config=config, model_dir=model_dir)
    
    # 전이 학습을 위해 COCO 가중치 로드 (최종 레이어 제외)
    model.load_weights(coco_model_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

    print("헤드 레이어 학습...")
    model.train(train_dataset, val_dataset,
                learning_rate=config.LEARNING_RATE,
                epochs=5, # 예시: 실제로는 더 많은 에폭 필요
                layers='heads')
    
    print("모든 레이어 미세 조정 학습...")
    model.train(train_dataset, val_dataset,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=10, # 예시: 실제로는 더 많은 에폭 필요
                layers='all')
    
    print("Mask R-CNN 학습 완료.")
    return model.find_last() # 학습된 최신 가중치 경로 반환

def extract_features(inference_config, dataset, model_path):
    """학습된 Mask R-CNN을 사용하여 특징 벡터를 추출합니다."""
    print("\n--- 2단계 (준비): 이미지 특징 벡터 추출 시작 ---")
    model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=os.path.dirname(model_path))
    model.load_weights(model_path, by_name=True)

    # ResNet 백본의 마지막 출력을 특징으로 사용하기 위한 새로운 모델 정의
    # 'res5c_out'은 ResNet-101의 마지막 컨볼루션 블록의 출력 레이어 이름입니다.
    feature_extractor = Model(inputs=model.keras_model.input,
                              outputs=model.keras_model.get_layer('res5c_out').output)
    
    features = []
    phase_labels = []
    
    for image_id in dataset.image_ids:
        image = dataset.load_image(image_id)
        # 이미지를 모델 입력에 맞게 리사이즈 및 정규화
        molded_image, _, _, _, _ = utils.mold_inputs([image], inference_config)
        # 특징 추출
        feature_map = feature_extractor.predict(molded_image)
        # 특징 맵을 1D 벡터로 변환 (Global Average Pooling)
        vector = np.mean(feature_map, axis=(1, 2)).squeeze()
        
        features.append(vector)
        phase_labels.append(dataset.load_phase_label(image_id))
        
        if (image_id + 1) % 100 == 0:
            print(f"{image_id + 1} / {len(dataset.image_ids)} 이미지 처리 완료")

    print(f"총 {len(features)}개의 특징 벡터 추출 완료.")
    return np.array(features), np.array(phase_labels)

def train_phase_classifier(X_train, y_train, X_val, y_val, num_phases, model_save_path):
    #"""추출된 특징 벡터로 수술 단계 분류기를 학습합니다."""
    print("\n--- 2단계: 수술 단계 분류기 학습 시작 ---")
    
    # 라벨을 원-핫 인코딩으로 변환
    y_train_cat = to_categorical(y_train, num_classes=num_phases)
    y_val_cat = to_categorical(y_val, num_classes=num_phases)
    
    input_shape = (X_train.shape[1],)
    
    # 간단한 MLP 모델 정의
    input_tensor = Input(shape=input_shape)
    x = Dense(512, activation='relu')(input_tensor)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_tensor = Dense(num_phases, activation='softmax')(x)
    
    classifier = Model(inputs=input_tensor, outputs=output_tensor)
    
    classifier.compile(optimizer=Adam(lr=0.0001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
    
    print(classifier.summary())
    
    classifier.fit(X_train, y_train_cat,
                   validation_data=(X_val, y_val_cat),
                   epochs=30, # 예시: 실제로는 더 많은 에폭 필요
                   batch_size=32)
    
    classifier.save(model_save_path)
    print(f"단계 분류기 모델 저장 완료: {model_save_path}")
    return classifier

# --- 3. 통합 추론 파이프라인 클래스 ---

class SurgicalPipeline:
    """Mask R-CNN과 단계 분류기를 통합하여 추론을 수행하는 클래스"""
    def __init__(self, mrcnn_config, mrcnn_weights_path, phase_classifier_path):
        print("\n--- 통합 파이프라인 초기화 ---")
        # 1. Mask R-CNN 모델 로드 (추론 모드)
        self.mrcnn_model = modellib.MaskRCNN(mode="inference", config=mrcnn_config,
                                             model_dir=os.path.dirname(mrcnn_weights_path))
        self.mrcnn_model.load_weights(mrcnn_weights_path, by_name=True)
        print("Mask R-CNN 가중치 로드 완료.")

        # 2. 특징 추출기 모델 정의
        self.feature_extractor = Model(inputs=self.mrcnn_model.keras_model.input,
                                       outputs=self.mrcnn_model.keras_model.get_layer('res5c_out').output)
        print("특징 추출기 모델 정의 완료.")
        
        # 3. 단계 분류기 모델 로드
        from keras.models import load_model
        self.phase_classifier = load_model(phase_classifier_path)
        print("단계 분류기 모델 로드 완료.")
        
        self.mrcnn_config = mrcnn_config

    def predict(self, image):
        """이미지 한 장에 대해 도구 인식과 단계 예측을 모두 수행합니다."""
        # 도구 인식 (Mask R-CNN)
        tool_results = self.mrcnn_model.detect([image], verbose=0)[0]
        
        # 단계 예측을 위한 특징 추출
        molded_image, _, _, _, _ = utils.mold_inputs([image], self.mrcnn_config)
        feature_map = self.feature_extractor.predict(molded_image)
        feature_vector = np.mean(feature_map, axis=(1, 2))
        
        # 단계 예측 (Phase Classifier)
        phase_prediction_probs = self.phase_classifier.predict(feature_vector)[0]
        predicted_phase_id = np.argmax(phase_prediction_probs)
        
        return tool_results, predicted_phase_id, phase_prediction_probs

# --- 4. 메인 실행 로직 ---
def main():
    # --- 경로 설정 ---
    ROOT_DIR = Path.cwd()
    MODEL_DIR = ROOT_DIR / "logs"
    COCO_MODEL_PATH = ROOT_DIR / "mask_rcnn_coco.h5"
    PHASE_CLASSIFIER_PATH = ROOT_DIR / "phase_classifier.h5"

    base_data_path = ROOT_DIR.parent / "dataset/dataset001/179.6개암 최소침습수술 AI학습데이터/01.데이터/1.Training/"
    phase_directory_path =ROOT_DIR / "output_phase_labeling5/"
    
    # --- 데이터셋 준비 ---
    parser = SurgicalDatasetParser(str(base_data_path), str(phase_directory_path))
    
    dataset_items = list(parser.prepare_dataset_for_keras())
    if not dataset_items: 
        print("오류: 처리된 데이터가 없어 프로그램을 종료합니다")
        return

    np.random.seed(42)
    np.random.shuffle(dataset_items)
    val_split_idx = int(len(dataset_items) * 0.9)
    train_items = dataset_items[:val_split_idx]
    val_items = dataset_items[val_split_idx:]

    dataset_train = SurgicalDataset(); dataset_train.load_surgical_data(parser, train_items, "train"); dataset_train.prepare()
    dataset_val = SurgicalDataset(); dataset_val.load_surgical_data(parser, val_items, "val"); dataset_val.prepare()

    # --- 설정 ---
    config = SurgicalConfig()
    config.NUM_CLASSES = 1 + len(parser.tool_categories)
    config.NUM_PHASES = parser.num_phases
    config.STEPS_PER_EPOCH = len(train_items) // config.IMAGES_PER_GPU
    config.VALIDATION_STEPS = len(val_items) // config.IMAGES_PER_GPU if val_items else 1
    #config.display()
    
    config.IMAGE_META_SIZE = 1 + 3 + 3 + 4 + 1 + config.NUM_CLASSES
    
    print("\n [수정완료]동적으로IMAGE_META_SIZE를 재계산 했습니다. ")
    config.display()
    
    # --- 1단계: Mask R-CNN 학습 ---
    # 이미 학습된 가중치가 있다면 이 단계를 건너뛸 수 있습니다.
    mrcnn_weights_path = train_mask_rcnn(config, dataset_train, dataset_val, str(MODEL_DIR),str(COCO_MODEL_PATH))
    # mrcnn_weights_path = "/path/to/your/trained/weights.h5" # 직접 경로 지정도 가능

    # --- 2단계: 특징 추출 및 분류기 학습 ---
    class InferenceConfig(SurgicalConfig):
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    inference_config = InferenceConfig()
    inference_config.NUM_CLASSES = config.NUM_CLASSES
    inference_config.NUM_PHASES = config.NUM_PHASES
    
    X_train, y_train = extract_features(inference_config, dataset_train, mrcnn_weights_path)
    X_val, y_val = extract_features(inference_config, dataset_val, mrcnn_weights_path)
    
    train_phase_classifier(X_train, y_train, X_val, y_val, parser.num_phases,str(PHASE_CLASSIFIER_PATH))

    # --- 3단계: 통합 파이프라인으로 추론 테스트 ---
    pipeline = SurgicalPipeline(inference_config, mrcnn_weights_path, str(PHASE_CLASSIFIER_PATH))
    
    # 검증 데이터셋에서 임의의 이미지로 테스트
    if val_items:
        image_id = np.random.choice(dataset_val.image_ids)
        image = dataset_val.load_image(image_id)
        true_phase_id = dataset_val.load_phase_label(image_id)
        
        tool_results, predicted_phase_id, phase_probs = pipeline.predict(image)
        
        predicted_phase_name = parser.idx_to_phase.get(predicted_phase_id, "N/A")
        true_phase_name = parser.idx_to_phase.get(true_phase_id, "N/A")
        
        print("\n--- 최종 추론 결과 ---")
        print(f"이미지: {dataset_val.image_reference(image_id)}")
        print(f"실제 수술 단계: {true_phase_name}")
        print(f"예측된 수술 단계: {predicted_phase_name} (신뢰도: {phase_probs[predicted_phase_id]:.2f})")
        
        # 도구 인식 결과 시각화
        visualize.display_instances(image, tool_results['rois'], tool_results['masks'],
                                    tool_results['class_ids'], dataset_val.class_names,
                                    tool_results['scores'],
                                    title=f"Predicted Phase: {predicted_phase_name}")
        plt.show()


In [4]:
if __name__ == '__main__':
    main()

발견된 수술 도구 카테고리: 26개
 ID 1: Bowel Graspers (straight) (AESCULAP) (GRSL-BWAE)
 ID 2: Dissector (Artisential) (DSTL-ATUK)
 ID 3: Dissector (kelly) (AESCULAP) (DSTL-KLAE)
 ID 4: Drain (JPCL-UCUK)
 ID 5: Endobulldog Clamp (CLML-BDUK)
 ID 6: Gauze (GAUL-UCUK)
 ID 7: Gauze Ball (GAUL-BAUK)
 ID 8: Graspers (curved) (KARL STORZ) (GRSL-C2KS)
 ID 9: Graspers (curved) (KARL STORZ) (GRSL-CIKS)
 ID 10: Graspers (duckbill) (GRSL-DBUK)
 ID 11: Graspers (straight) (KARL STORZ) (GRSL-STKS)
 ID 12: Harmonic Scalpel (ETHICON) (HMSL-UCET)
 ID 13: Irrigator (IRGL-UCUK)
 ID 14: Ligasure (COVIDIEN) (LGSL-MACV)
 ID 15: Linear Stapler (EGSL-UCUK)
 ID 16: Linear Stapler (ETHICON) (STPL-UCET)
 ID 17: Liver retractor (LVRL-UCUK)
 ID 18: Needle Holder (AESCULAP) (NDHL-UCAE)
 ID 19: Obturator (OBTL-UCUK)
 ID 20: Polymer Clip Applier (CLAL-UCUK)
 ID 21: Spatula Suction (ETHICON) (SPSL-UCET)
 ID 22: Specimen Retrieval Bag (RTBL-UCUK)
 ID 23: Suction Irrigator (ETHICON) (SIRL-UCET)
 ID 24: Suction Irrigator (RDSL-UCUK)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
1093/4395 [======>.......................] - ETA: 18:56:01 - loss: 1.3147 - rpn_class_loss: 0.0075 - rpn_bbox_loss: 0.3657 - mrcnn_class_loss: 0.1696 - mrcnn_bbox_loss: 0.3335 - mrcnn_mask_loss: 0.4383

Process ForkPoolWorker-12:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Process ForkPoolWorker-9:
Process ForkPoolWorker-4:
Process ForkPoolWorker-15:
Process ForkPoolWorker-1:
Process ForkPoolWorker-13:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-7:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
 

KeyboardInterrupt: 

In [50]:
import mrcnn
import os
print(os.path.dirname(mrcnn.__file__))
print(mrcnn.__file__)

/usr/local/lib/python3.6/dist-packages/mrcnn
/usr/local/lib/python3.6/dist-packages/mrcnn/__init__.py


In [51]:
cd /usr/local/lib/python3.6/dist-packages/mrcnn

/usr/local/lib/python3.6/dist-packages/mrcnn


In [18]:
# Keras/TF 버전으로 수정: 필요한 라이브러리 임포트
import os
import sys
import json
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

# Keras 및 TensorFlow 관련 라이브러리
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

# lxml 임포트
try:
    import lxml.etree as etree
except ImportError:
    import xml.etree.ElementTree as etree
    print("경고: lxml 라이브러리를 찾을 수 없어 표준 xml.etree.ElementTree를 사용합니다.")

# Matterport Mask R-CNN 라이브러리 임포트
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# --- 1. 데이터 파싱 및 설정 클래스 (이전과 거의 동일) ---
# 이전 답변에서 제공된 SurgicalDatasetParser, SurgicalDataset, SurgicalConfig 클래스를 그대로 사용합니다.
# 생략된 코드는 첨부된 파일 [1] 또는 이전 답변을 참고해주세요.
class SurgicalDatasetParser:
    # ... 이전 코드와 동일 ... [1]
    def __init__(self, base_path, phase_directory_path):
        self.base_path = Path(base_path)
        self.label_path = self.base_path/"라벨링데이터"/"클래스1.(수술도구 이미지)"/"01.원위부위절제술"
        self.image_path = self.base_path/"원천데이터"/"클래스1.(수술도구 이미지)"/"01.원위부위절제술"
        self.tool_categories = self._discover_tool_categories()
        self.class_to_idx = {cat['name']: cat['id'] for cat in self.tool_categories}
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}
        print(f"발견된 수술 도구 카테고리: {len(self.tool_categories)}개")
        for cat in self.tool_categories: print(f" ID {cat['id']}: {cat['name']}")
        self.image_stem_to_phase_map = self._load_phase_data(phase_directory_path)
        if self.image_stem_to_phase_map:
            self.phases = sorted(list(set(self.image_stem_to_phase_map.values())))
            self.phase_to_idx = {name: i for i, name in enumerate(self.phases)}
            self.idx_to_phase = {i: name for i, name in enumerate(self.phases)}
            self.num_phases = len(self.phases)
            print(f"발견된 수술 단계: {self.num_phases}개 - {self.phases}")
        else:
            self.phases, self.phase_to_idx, self.idx_to_phase, self.num_phases = [], {}, {}, 0
    def _discover_tool_categories(self):
        categories = []
        if self.label_path.exists():
            tool_folders = sorted([f for f in self.label_path.iterdir() if f.is_dir()])
            categories = [{"id": idx + 1, "name": folder.name} for idx, folder in enumerate(tool_folders)]
        return categories
    def _load_phase_data(self, phase_directory_path):
        image_stem_to_phase = {}
        if not phase_directory_path: return image_stem_to_phase
        phase_dir = Path(phase_directory_path)
        if not phase_dir.is_dir(): return image_stem_to_phase
        for json_file_path in phase_dir.glob("*.json"):
            try:
                with open(json_file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                for entry in data:
                    if isinstance(entry, dict) and "filename" in entry and "phase" in entry:
                        image_stem_to_phase[Path(entry["filename"]).stem] = entry["phase"]
            except Exception: continue
        return image_stem_to_phase
    def prepare_dataset_for_keras(self):
        dataset_items = []
        total_items = sum(1 for tool_dir in self.label_path.iterdir() if tool_dir.is_dir())
        processed_items = 0
        print(f"전체 항목 수 : {total_items}개")
        for tool_dir in self.label_path.iterdir():
            if not tool_dir.is_dir(): continue
            for xml_file in tool_dir.glob("*.json"):
                annotation = self._parse_xml_annotation(xml_file)
                if not annotation or not annotation.get('objects'): continue
                image_file = self._find_matching_image(xml_file.stem, tool_dir.name)
                if not image_file: continue
                phase_label = self.image_stem_to_phase_map.get(xml_file.stem)
                if phase_label is None or phase_label not in self.phase_to_idx: continue
                dataset_items.append({
                    'image_path': str(image_file),
                    'width': annotation['size']['width'], 'height': annotation['size']['height'],
                    'objects': annotation['objects'],
                    'phase_id': self.phase_to_idx[phase_label]
                })
                processed_items += 1
                print(f"진행 중 : {processed_items} / {total_items} 항목 처리 완료")
                                               
        print(f"총 {len(dataset_items)}개 학습/검증용 아이템 준비 완료")
        return dataset_items
                                               
    def _parse_xml_annotation(self, xml_file):
        try:
            tree = etree.parse(str(xml_file))
            size = tree.find('size')
            objects = []
            for obj in tree.findall('object'):
                class_name = obj.find('name').text
                if class_name not in self.class_to_idx: continue
                points_elem = obj.find('points')
                if points_elem is not None:
                    points = [{'x': float(x.text), 'y': float(y.text)} for x, y in zip(points_elem.findall('x'), points_elem.findall('y'))]
                    if points: objects.append({'name': class_name, 'points': points})
            return {'size': {'width': int(size.find('width').text), 'height': int(size.find('height').text)}, 'objects': objects}
        except Exception: return None
    def _find_matching_image(self, stem, folder):
        for ext in ['.png', '.jpg', '.jpeg']:
            img_path = self.image_path / folder / (stem + ext)
            if img_path.exists():
                return img_path
        return None
class SurgicalDataset(utils.Dataset):
    # ... 이전 코드와 동일 ... [1]
    def load_surgical_data(self, parser, dataset_items, dataset_type):
        self.parser = parser
        for cat in self.parser.tool_categories: self.add_class("surgical_tool", cat['id'], cat['name'])
        for i, item in enumerate(dataset_items):
            self.add_image("surgical_tool", image_id=i, path=item['image_path'], width=item['width'], height=item['height'], polygons=item['objects'], phase_id=item['phase_id'])
        print(f"'{dataset_type}' 데이터셋에 {len(dataset_items)}개 이미지를 로드했습니다.")
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        masks, class_ids = [], []
        for obj in info["polygons"]:
            points = np.array([[p['x'], p['y']] for p in obj['points']], dtype=np.int32)
            mask = np.zeros([info["height"], info["width"]], dtype=np.uint8)
            cv2.fillPoly(mask, [points], 1)
            masks.append(mask)
            class_ids.append(self.parser.class_to_idx[obj['name']])
        return (np.stack(masks, axis=-1).astype(bool), np.array(class_ids, dtype=np.int32)) if masks else super(self.__class__, self).load_mask(image_id)
    def image_reference(self, image_id): return self.image_info[image_id]["path"]
    def load_phase_label(self, image_id): return self.image_info[image_id]['phase_id']

class SurgicalConfig(Config):
    # ... 이전 코드와 동일 ... [1]
    NAME = "surgical_tool_phase"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    LEARNING_RATE = 0.001

# --- 2. 모델 학습 및 특징 추출, 분류기 학습 함수 정의 ---

def train_mask_rcnn(config, train_dataset, val_dataset, model_dir, coco_model_path):
    """1단계: Mask R-CNN 모델을 학습시킵니다."""
    print("\n--- 1단계: Mask R-CNN (특징 추출기) 학습 시작 ---")
    model = modellib.MaskRCNN(mode="training", config=config, model_dir=model_dir)
    
    # 전이 학습을 위해 COCO 가중치 로드 (최종 레이어 제외)
    model.load_weights(coco_model_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

    print("헤드 레이어 학습...")
    model.train(train_dataset, val_dataset,
                learning_rate=config.LEARNING_RATE,
                epochs=5, # 예시: 실제로는 더 많은 에폭 필요
                layers='heads')
    
    print("모든 레이어 미세 조정 학습...")
    model.train(train_dataset, val_dataset,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=10, # 예시: 실제로는 더 많은 에폭 필요
                layers='all')
    
    print("Mask R-CNN 학습 완료.")
    return model.find_last() # 학습된 최신 가중치 경로 반환

def extract_features(inference_config, dataset, model_path):
    """학습된 Mask R-CNN을 사용하여 특징 벡터를 추출합니다."""
    print("\n--- 2단계 (준비): 이미지 특징 벡터 추출 시작 ---")
    model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=os.path.dirname(model_path))
    model.load_weights(model_path, by_name=True)

    # ResNet 백본의 마지막 출력을 특징으로 사용하기 위한 새로운 모델 정의
    # 'res5c_out'은 ResNet-101의 마지막 컨볼루션 블록의 출력 레이어 이름입니다.
    feature_extractor = Model(inputs=model.keras_model.input,
                              outputs=model.keras_model.get_layer('res5c_out').output)
    
    features = []
    phase_labels = []
    
    for image_id in dataset.image_ids:
        image = dataset.load_image(image_id)
        # 이미지를 모델 입력에 맞게 리사이즈 및 정규화
        molded_image, _, _, _, _ = utils.mold_inputs([image], inference_config)
        # 특징 추출
        feature_map = feature_extractor.predict(molded_image)
        # 특징 맵을 1D 벡터로 변환 (Global Average Pooling)
        vector = np.mean(feature_map, axis=(1, 2)).squeeze()
        
        features.append(vector)
        phase_labels.append(dataset.load_phase_label(image_id))
        
        if (image_id + 1) % 100 == 0:
            print(f"{image_id + 1} / {len(dataset.image_ids)} 이미지 처리 완료")

    print(f"총 {len(features)}개의 특징 벡터 추출 완료.")
    return np.array(features), np.array(phase_labels)

def train_phase_classifier(X_train, y_train, X_val, y_val, num_phases, model_save_path):
    #"""추출된 특징 벡터로 수술 단계 분류기를 학습합니다."""
    print("\n--- 2단계: 수술 단계 분류기 학습 시작 ---")
    
    # 라벨을 원-핫 인코딩으로 변환
    y_train_cat = to_categorical(y_train, num_classes=num_phases)
    y_val_cat = to_categorical(y_val, num_classes=num_phases)
    
    input_shape = (X_train.shape[1],)
    
    # 간단한 MLP 모델 정의
    input_tensor = Input(shape=input_shape)
    x = Dense(512, activation='relu')(input_tensor)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_tensor = Dense(num_phases, activation='softmax')(x)
    
    classifier = Model(inputs=input_tensor, outputs=output_tensor)
    
    classifier.compile(optimizer=Adam(lr=0.0001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
    
    print(classifier.summary())
    
    classifier.fit(X_train, y_train_cat,
                   validation_data=(X_val, y_val_cat),
                   epochs=30, # 예시: 실제로는 더 많은 에폭 필요
                   batch_size=32)
    
    classifier.save(model_save_path)
    print(f"단계 분류기 모델 저장 완료: {model_save_path}")
    return classifier

# --- 3. 통합 추론 파이프라인 클래스 ---

class SurgicalPipeline:
    """Mask R-CNN과 단계 분류기를 통합하여 추론을 수행하는 클래스"""
    def __init__(self, mrcnn_config, mrcnn_weights_path, phase_classifier_path):
        print("\n--- 통합 파이프라인 초기화 ---")
        # 1. Mask R-CNN 모델 로드 (추론 모드)
        self.mrcnn_model = modellib.MaskRCNN(mode="inference", config=mrcnn_config,
                                             model_dir=os.path.dirname(mrcnn_weights_path))
        self.mrcnn_model.load_weights(mrcnn_weights_path, by_name=True)
        print("Mask R-CNN 가중치 로드 완료.")

        # 2. 특징 추출기 모델 정의
        self.feature_extractor = Model(inputs=self.mrcnn_model.keras_model.input,
                                       outputs=self.mrcnn_model.keras_model.get_layer('res5c_out').output)
        print("특징 추출기 모델 정의 완료.")
        
        # 3. 단계 분류기 모델 로드
        from keras.models import load_model
        self.phase_classifier = load_model(phase_classifier_path)
        print("단계 분류기 모델 로드 완료.")
        
        self.mrcnn_config = mrcnn_config

    def predict(self, image):
        """이미지 한 장에 대해 도구 인식과 단계 예측을 모두 수행합니다."""
        # 도구 인식 (Mask R-CNN)
        tool_results = self.mrcnn_model.detect([image], verbose=0)[0]
        
        # 단계 예측을 위한 특징 추출
        molded_image, _, _, _, _ = utils.mold_inputs([image], self.mrcnn_config)
        feature_map = self.feature_extractor.predict(molded_image)
        feature_vector = np.mean(feature_map, axis=(1, 2))
        
        # 단계 예측 (Phase Classifier)
        phase_prediction_probs = self.phase_classifier.predict(feature_vector)[0]
        predicted_phase_id = np.argmax(phase_prediction_probs)
        
        return tool_results, predicted_phase_id, phase_prediction_probs

# --- 4. 메인 실행 로직 ---
def main():
    # --- 경로 설정 ---
    ROOT_DIR = Path.cwd()
    MODEL_DIR = ROOT_DIR / "logs"
    COCO_MODEL_PATH = ROOT_DIR / "mask_rcnn_coco.h5"
    PHASE_CLASSIFIER_PATH = ROOT_DIR / "phase_classifier.h5"

    base_data_path = Path("../dataset/dataset001/179.6개암 최소침습수술 AI학습데이터/01.데이터/1.Training/")
    phase_directory_path = Path("./output_phase_labeling5/")
    
    # --- 데이터셋 준비 ---
    parser = SurgicalDatasetParser(base_data_path, phase_directory_path)
    dataset_items = parser.prepare_dataset_for_keras()
    if not dataset_items: return

    np.random.seed(42)
    np.random.shuffle(dataset_items)
    val_split_idx = int(len(dataset_items) * 0.9)
    train_items = dataset_items[:val_split_idx]
    val_items = dataset_items[val_split_idx:]

    dataset_train = SurgicalDataset(); dataset_train.load_surgical_data(parser, train_items, "train"); dataset_train.prepare()
    dataset_val = SurgicalDataset(); dataset_val.load_surgical_data(parser, val_items, "val"); dataset_val.prepare()

    # --- 설정 ---
    config = SurgicalConfig()
    config.NUM_CLASSES = 1 + len(parser.tool_categories)
    config.NUM_PHASES = parser.num_phases
    config.STEPS_PER_EPOCH = len(train_items) // config.IMAGES_PER_GPU
    config.VALIDATION_STEPS = len(val_items) // config.IMAGES_PER_GPU if val_items else 1
    config.display()
    
    # --- 1단계: Mask R-CNN 학습 ---
    # 이미 학습된 가중치가 있다면 이 단계를 건너뛸 수 있습니다.
    mrcnn_weights_path = train_mask_rcnn(config, dataset_train, dataset_val, MODEL_DIR, COCO_MODEL_PATH)
    # mrcnn_weights_path = "/path/to/your/trained/weights.h5" # 직접 경로 지정도 가능

    # --- 2단계: 특징 추출 및 분류기 학습 ---
    class InferenceConfig(SurgicalConfig):
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    inference_config = InferenceConfig()
    inference_config.NUM_CLASSES = config.NUM_CLASSES
    inference_config.NUM_PHASES = config.NUM_PHASES
    
    X_train, y_train = extract_features(inference_config, dataset_train, mrcnn_weights_path)
    X_val, y_val = extract_features(inference_config, dataset_val, mrcnn_weights_path)
    
    train_phase_classifier(X_train, y_train, X_val, y_val, parser.num_phases, PHASE_CLASSIFIER_PATH)

    # --- 3단계: 통합 파이프라인으로 추론 테스트 ---
    pipeline = SurgicalPipeline(inference_config, mrcnn_weights_path, PHASE_CLASSIFIER_PATH)
    
    # 검증 데이터셋에서 임의의 이미지로 테스트
    if val_items:
        image_id = np.random.choice(dataset_val.image_ids)
        image = dataset_val.load_image(image_id)
        true_phase_id = dataset_val.load_phase_label(image_id)
        
        tool_results, predicted_phase_id, phase_probs = pipeline.predict(image)
        
        predicted_phase_name = parser.idx_to_phase.get(predicted_phase_id, "N/A")
        true_phase_name = parser.idx_to_phase.get(true_phase_id, "N/A")
        
        print("\n--- 최종 추론 결과 ---")
        print(f"이미지: {dataset_val.image_reference(image_id)}")
        print(f"실제 수술 단계: {true_phase_name}")
        print(f"예측된 수술 단계: {predicted_phase_name} (신뢰도: {phase_probs[predicted_phase_id]:.2f})")
        
        # 도구 인식 결과 시각화
        visualize.display_instances(image, tool_results['rois'], tool_results['masks'],
                                    tool_results['class_ids'], dataset_val.class_names,
                                    tool_results['scores'],
                                    title=f"Predicted Phase: {predicted_phase_name}")
        plt.show()

if __name__ == '__main__':
    main()

발견된 수술 도구 카테고리: 26개
 ID 1: Bowel Graspers (straight) (AESCULAP) (GRSL-BWAE)
 ID 2: Dissector (Artisential) (DSTL-ATUK)
 ID 3: Dissector (kelly) (AESCULAP) (DSTL-KLAE)
 ID 4: Drain (JPCL-UCUK)
 ID 5: Endobulldog Clamp (CLML-BDUK)
 ID 6: Gauze (GAUL-UCUK)
 ID 7: Gauze Ball (GAUL-BAUK)
 ID 8: Graspers (curved) (KARL STORZ) (GRSL-C2KS)
 ID 9: Graspers (curved) (KARL STORZ) (GRSL-CIKS)
 ID 10: Graspers (duckbill) (GRSL-DBUK)
 ID 11: Graspers (straight) (KARL STORZ) (GRSL-STKS)
 ID 12: Harmonic Scalpel (ETHICON) (HMSL-UCET)
 ID 13: Irrigator (IRGL-UCUK)
 ID 14: Ligasure (COVIDIEN) (LGSL-MACV)
 ID 15: Linear Stapler (EGSL-UCUK)
 ID 16: Linear Stapler (ETHICON) (STPL-UCET)
 ID 17: Liver retractor (LVRL-UCUK)
 ID 18: Needle Holder (AESCULAP) (NDHL-UCAE)
 ID 19: Obturator (OBTL-UCUK)
 ID 20: Polymer Clip Applier (CLAL-UCUK)
 ID 21: Spatula Suction (ETHICON) (SPSL-UCET)
 ID 22: Specimen Retrieval Bag (RTBL-UCUK)
 ID 23: Suction Irrigator (ETHICON) (SIRL-UCET)
 ID 24: Suction Irrigator (RDSL-UCUK)

KeyboardInterrupt: 

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

ModuleNotFoundError: No module named 'tensorflow.python.client'

In [9]:
import tensorflow as tf
print(tf._version_)
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

AttributeError: module 'tensorflow' has no attribute '_version_'

In [5]:
pip list

Package                Version
---------------------- -----------
absl-py                1.4.0
argon2-cffi            21.3.0
argon2-cffi-bindings   21.2.0
asn1crypto             0.24.0
astor                  0.8.1
async-generator        1.10
attrs                  22.2.0
backcall               0.2.0
bleach                 4.1.0
cached-property        1.5.2
cffi                   1.15.1
click                  8.0.4
cryptography           2.1.4
cycler                 0.11.0
dataclasses            0.8
decorator              4.4.2
defusedxml             0.7.1
entrypoints            0.4
Flask                  2.0.3
gast                   0.6.0
google-pasta           0.2.0
grpcio                 1.48.2
h5py                   2.10.0
idna                   2.6
imageio                2.15.0
importlib-metadata     4.8.3
importlib-resources    5.4.0
intel-cmplr-lib-ur     2024.2.1
intel-openmp           2024.2.1
ipykernel              5.5.6
ipython                7.16.3
ipython-genutils       0.2

In [9]:
from IPython.display import display, HTML

In [15]:
display(HTML("<style>.container{width:100%!important;}</style>"))

In [3]:
pip uninstall -y tensorflow  

Found existing installation: tensorflow 1.13.1
Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# 위 명령시 gpu 인식 못하고 CPU 만 인식 - tensor flow 와 tesnorflow-gpu 의 버전이 달라서 생기는 오류로 예상 
# 위 오류 해결후 동작하면 잘 되길..


AttributeError: module 'tensorflow' has no attribute 'compat'

In [ ]:
from keras import backend as k
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
k.set_session(sess)

In [10]:
import sys

In [11]:
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [14]:
pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
Note: you may need to restart the kernel to use updated packages.
